<a id='TOP'></a>

### Data cleaning
[Exploring]
### Models
[Train Test Split](#TTS)  
[Decision Tree](#clf_DT)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('BusinessMasterFile.csv')
df.head()

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
msno.matrix(df)

In [ ]:
msno.heatmap(df)

In [ ]:
df.info()

In [ ]:
print(df.REVENUE_AMT.value_counts()[:5])
df.REVENUE_AMT.isna().value_counts()

In [ ]:
print(df.ASSET_AMT.value_counts()[:5])
print(df.ASSET_AMT.isna().sum())
print(df.ASSET_CD.value_counts()[:5])
print(df.ASSET_CD.isna().sum())

I am trying to decide whether or not to drop ASSEST_CD/ASSET_AMT and INCOME_AMT/INCOME_CD.

ASSET_AMT is continious data, rather than the coded data in ASSET_CD, so I will keep the AMTs reported over the prebinned. I can alter it myself if need be. Also there seems to be some discrepancy in values from the coded and the reported, which makes my decision. 

Tasks:
- Combine `city`, `street`, `state` into lat and longs. Drop `city`, `street`, `state`. This is the location of the headquarters and doesn't mean that the 501c has operations in that state. https://github.com/geopy/geopy
- Change datetime data into correct formats. `TAX_PERIOD`, `RULING_DATE`
- Make things categorical
- Investigate any connection between SORT_NAME and GROUP EXEMPTION NUM and AFFILCATION CODE [Connection?](#task1)
- Check the Ruling year after 1995 and see if any NTEE exisits.


<a id="task1"></a>

Sort Name Line is another name under which the organization does business.  Also used for trade names, chapter names, or local numbers for subordinate organizations of group rulings

Central - This code is used if the organization is a central type organization (no group exemption) of a National, Regional or Geographic grouping of organizations. 2 Intermediate - This code is used if the organization is an intermediate organization (no group exemption) of a National, Regional or Geographic grouping of organizations (such as a state headquarters of a national organization). 3 Independent - This code is used if the organization is an independent organization or an independent auxiliary 
(i.e., not affiliated with a National, Regional, or Geographic grouping of organizations). 6 Central - This code is used if the organization is a parent (group ruling) and is not a church or 501(c)(1) organization. 7 Intermediate - This code is used if the organization is a group exemption intermediate organization of a National, Regional or Geographic grouping of organizations. 8 Central - This code is used if the organization is a parent (group ruling) and is a church or 501(c)(1) organization. 9 Subordinate - This code is used if the organization is a subordinate in a group ruling

In [ ]:
df.drop(['ICO', 
         'RULING', 
         'ACT2', 
         'ACT3', 
         'ASSET_CD', 
         'INCOME_CD', 
         'LEVEL4', 
         'TAX_PERIOD'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
msno.matrix(df)

There is a clear pattern of missing values from `ASSET_AMT`,`INCOME_AMT`, and `REVENUE_AMT`. I will drop values from one column, extending down the rows and then will recheck the patterns.

In [ ]:
df = df.dropna(subset=['ASSET_AMT'])
msno.matrix(df)

~370,000 entries were removed. 

In [ ]:
df.isna().sum()

The last 100K from `REVENUE_AMT` will be dropped as well. I will combine the `SORT_NAME` column with the `NAME` column since they both come from the same field, just a different line. `NTEE_CD` will be addressed seperately as well.

In [ ]:
df = df.dropna(subset=['REVENUE_AMT'])

In [ ]:
df.isna().sum()

In [ ]:
df.SORT_NAME = df.SORT_NAME.fillna(value = '') #Replaces np.NaN values with something else, string + np.NaN = np.NaN
df['NAME_FULL'] = df.NAME + ' ' + df.SORT_NAME #New column
df.drop(['NAME', 'SORT_NAME'], axis=1, inplace=True) #Getting rid of the old
df.head()

In [ ]:
df.isna().sum()

My target is going to `ACTIVITY`/`NTEE_CD`. The values in `ACTIVITY` are 9 characters long, each set of three indicating what kind of the organization it is There are some that needed more than one set of three to define their organization, others only have one. Planning to use only the first activity code. Also I have to keep in mind, there are ones that could apply to different targets. As expected, the data itself is not that clean. Effort will be made to tidy it up.

The NTEE_CD code was introduced to replace the three digit code after **(CHECK if not 1994/1996)** tax year 1995. I will make my own mapping from the old system to the new system.  

I will run three models; a pre-1995 target, post-1995 target, and the entire dataset with a semi-converted target. The mapping that I create will is an area of potential error. This is a place of further improvement on future iterations of this project. 

Lets now grab some quick masks on the smaller subsets to be used later.

In [ ]:
df_post = df
post_index = df_post.NTEE_CD.notna() #mask for all the non-profits with the new coding

post_index

In [ ]:
print(df.RULEYEAR[post_index].value_counts()[:5])

Looks good. **ASK JEFF ABOUT THIS**

In [ ]:
df.loc[(df.NTEE_CD.notna()) & (df.ACTIVITY == 0)].info()

In [ ]:
df.NTEE_CD.notna().sum()

In [ ]:
df.NTEE_CD.isna().sum()

In [ ]:
df.ACTIVITY.value_counts()[:10]

There is something wrong with the above `.value_counts()`. The Activity Codes provided by the IRS have leading zeroes. Since there wasn't an attempt to control for it them when orginally reading the files, I want further to investigate if the `ACTIVITY` lines contain 9 characters. If they don't, I will adjust them accordingly.

In [ ]:
temp = df.ACTIVITY

In [ ]:
length = []
for x in temp:
    length.append(len(str(x)))
pd.Series(length).value_counts()

As expected, adjustments are need. Make a small function that will adjust the `ACTIVITY` input to match the 9 character length expected.

In [ ]:
def adj_len(num):
    string = str(num)
    
    if len(string) >= 7:
        
        if len(string) == 9:
            return int(string[0:3])
        if len(string) == 8:
            return int(string[0:2])
        if len(string) == 7:
            return int(string[0:1])
    else:
        return num
    

In [ ]:
adj_len(123456789)

Tester set for a encoding that I will do.

In [ ]:
small = df.loc[(df.NTEE_CD.isna()) & (df.ACTIVITY != 0)][:100]

In [ ]:
def code_NTEE_V1(y):
    
    x = adj_len(y)
    
    list_a =list(range(60,120))
    list_b =list(range(30,60)) + [540, 914]
    list_c =list(range(350,380)) + [529]
    list_d =[913]
    list_e =list(range(150,180)) + [531] + list(range(541,544))
    list_f = []
    list_g = []
    list_h = []
    list_i =list(range(460,466)) + [406, 526, 527, 528]
    list_j =list(range(120,150)) + list(range(200,230)) + [920]
    list_k =list(range(230,250)) + [916]
    list_l =list(range(380,400)) + [401]
    list_m =[902]
    list_n =list(range(280,320))
    list_o =list(range(320,350))
    list_p =list(range(563,576)) + [917, 918]
    list_q =list(range(510,514)) + [518, 519, 520, 522, 910, 911, 912]
    list_r =list(range(430,450)) + list(range(480,510)) + [261, 405, 481, 482, 534, 535, 924]
    list_s =list(range(408,430)) + [404, 524, 533, 903, 919, 921]
    list_t =list(range(600,604)) + list(range(560,563))
    list_u =list(range(180,200)) 
    list_v = []
    list_w =list(range(250,253)) + list(range(536,540)) + list(range(905,909)) + [262, 263, 400, 402, 403, 407, 514, 515, 516, 521, 523, 525, 530 , 532, 559, 915]
    list_x =list(range(1,30)) + [517]
    list_y =list(range(264,280)) + [253, 254, 259, 260, 900, 901, 922]
    list_z = []

    if x in list_a:
        return 'A'
    if x in list_b:
        return 'B'
    if x in list_c:
        return 'C'
    if x in list_d:
        return 'D'
    if x in list_e:
        return 'E'
    if x in list_f:
        return 'F'
    if x in list_g:
        return 'G'
    if x in list_h:
        return 'H'
    if x in list_i:
        return 'I'
    if x in list_j:
        return 'J'
    if x in list_k:
        return 'K'
    if x in list_l:
        return 'L' 
    if x in list_m:
        return 'M'
    if x in list_n:
        return 'N'
    if x in list_o:
        return 'O'
    if x in list_p:
        return 'P'
    if x in list_q:
        return 'Q'
    if x in list_r:
        return 'R'
    if x in list_s:
        return 'S'
    if x in list_t:
        return 'T'
    if x in list_u:
        return 'U'
    if x in list_v:
        return 'V'
    if x in list_w:
        return 'W'
    if x in list_x:
        return 'X'
    if x in list_y:
        return 'Y'
    if x in list_z:
        return 'Z'
    else:
        trouble.append(x)
        
    

In [ ]:
print(code_NTEE_V1(205000000))

Test something with a leading zero, or an `ACTIVITY` that does not have 9 characters 

In [ ]:
trouble = [] #Within my small data set, any codes that don't have a new label
small.ACT1.apply(code_NTEE_V1)
print(set(trouble))
print(len(trouble))

With the quick check, there are codes that were not coded. This was to be expected. Within the set above, none of these are defined by the IRS. I suspect this set to grown once applied to the larger dataset.  

Within the process, I will update the master list with proper categorizations through out. The mapping will be adjusted accordingly. Please see the documentation if further review is [desired](./irs_jan_2010.pdf) 

In [ ]:
small.NTEE_CD = small.ACT1.apply(code_NTEE_V1)
small.NTEE_CD.isna().sum()

When updating the new column, columns that don't have a mapping will be updated to be a `None` value that will be later removed.

Lets take care of the subset of rows that can not have any targets for.

In [ ]:
df.NTEE_CD.isna().sum()

In [ ]:
print(df[(df.NTEE_CD.isna()) & (df.ACT1 == 0)].shape)
df[(df.NTEE_CD.isna()) & (df.ACT1 == 0)].head()

In [ ]:
before = df.shape

mask = df.index[(df.NTEE_CD.isna()) & (df.ACT1 == 0)] #index numbers to drop
df = df.drop(mask)

after = df.shape

print(f'From {before} rows, down to {after} rows')

In [ ]:
trouble = [] #the code_NTEE(X) function has a list to check the missing variables
look = df.ACTIVITY.apply(code_NTEE_V1)
set(trouble)

At this point, I manually went back and updated my `code_NTEE` function. I updated lists and created a new list for undefined codes. Also, there were a few ambigious codings that I have decided to drop altogeter. These includes definitions such as "Indians (tribes, cultures, etc.)" , "Government instrumentality or agency", and "947(a)(2) trust". Further man hours are required to shift through the material to categorize them correctly.  

Using the updated `code_NTEE` function below, I will move forward with the data cleaning process. https://nccs.urban.org/publication/irs-activity-codes

In [ ]:
def code_NTEE_V2(y):
    
    x = adj_len(y)
    
    list_a =list(range(60,120))
    list_b =list(range(30,60)) + [540, 914]
    list_c =list(range(350,380)) + [529]
    list_d =[913]
    list_e =list(range(150,180)) + [531] + list(range(541,544))
    list_f = []
    list_g = []
    list_h = []
    list_i =list(range(460,466)) + [406, 526, 527, 528]
    list_j =list(range(120,150)) + list(range(200,230)) + [920]
    list_k =list(range(230,250)) + [916]
    list_l =list(range(380,400)) + [401]
    list_m =[902]
    list_n =list(range(280,320))
    list_o =list(range(320,350))
    list_p =list(range(563,576)) + [917, 918]
    list_q =list(range(510,514)) + [518, 519, 520, 522, 910, 911, 912]
    list_r =list(range(430,450)) + list(range(480,510)) + [261, 405, 481, 482, 534, 535, 924]
    list_s =list(range(408,430)) + [404, 524, 533, 903, 919, 921]
    list_t =list(range(600,604)) + list(range(560,563))
    list_u =list(range(180,200)) 
    list_v = []
    list_w =list(range(250,253)) + list(range(536,540)) + list(range(905,909)) + [262, 263, 400, 402, 403, 407, 514, 515, 516, 521, 523, 525, 530 , 532, 559, 915]
    list_x =list(range(1,30)) + [517]
    list_y =list(range(264,280)) + [253, 254, 259, 260, 900, 901, 922]
    list_z = []
    
    too_vague = [904, 909, 923] + list(range(925,999))
    not_defined = [256, 257, 466,469, 470, 556, 557, 999] + list(range(450,460)) + list(range(484, 509)) + list(range(576, 600)) + list(range(603,900)) + too_vague

    if x == 0:
        return None
    if x in list_a:
        return 'A'
    if x in list_b:
        return 'B'
    if x in list_c:
        return 'C'
    if x in list_d:
        return 'D'
    if x in list_e:
        return 'E'
    if x in list_f:
        return 'F'
    if x in list_g:
        return 'G'
    if x in list_h:
        return 'H'
    if x in list_i:
        return 'I'
    if x in list_j:
        return 'J'
    if x in list_k:
        return 'K'
    if x in list_l:
        return 'L' 
    if x in list_m:
        return 'M'
    if x in list_n:
        return 'N'
    if x in list_o:
        return 'O'
    if x in list_p:
        return 'P'
    if x in list_q:
        return 'Q'
    if x in list_r:
        return 'R'
    if x in list_s:
        return 'S'
    if x in list_t:
        return 'T'
    if x in list_u:
        return 'U'
    if x in list_v:
        return 'V'
    if x in list_w:
        return 'W'
    if x in list_x:
        return 'X'
    if x in list_y:
        return 'Y'
    if x in list_z:
        return 'Z'
    if x in not_defined:
        return None
    else:
        trouble.append(x)
        
    

In [ ]:
trouble = [] #the code_NTEE(X) function has a list to check the missing variables
look = df.ACTIVITY.apply(code_NTEE_V2)
trouble

There were no trouble codes anymore. We will merge the changes and tidy up the last few things wrong with the data.
Keep in mind that this step heavily effects the target outcome in the joint dataset.

In [ ]:
df.NTEE_CD.isna().sum()

In [ ]:
df.NTEE_CD.fillna(df.ACTIVITY.apply(code_NTEE_V2), inplace=True)

In [ ]:
print(df.NTEE_CD.isna().sum())
df[df.NTEE_CD.isna()].head()

Finally dropping the remaining ~11K rows that don't have a clear NTEE code.

In [ ]:
df = df.dropna(subset=['NTEE_CD'])
df.shape

In [ ]:
df.isna().sum()

Now to drop the very small subset of missing values.

In [ ]:
df.dropna(subset=['STATE', 'STREET'], inplace=True) #dropping the missing values
df.info()

Now taking a look at `NTEE_CD` column and getting the proper format.

In [ ]:
df.NTEE_CD.head()

Lets remove the more specific coding form the `NTEE` column, just leaving the one of the 26 categories.

In [ ]:
df.NTEE_CD = df.NTEE_CD.apply(lambda x: x[0:1])
df.NTEE_CD.head()

In [ ]:
df.ZIP = df.ZIP.apply(lambda x: str(x)[0:5])

Finally, lets drop the last few rows that won't be used and change the date types for the columns that need it.

In [ ]:
df.columns

Not forgetting we have subsets of our data, grabbing the masks we created before cleaning the data.

In [ ]:
df.isna().sum()

In [ ]:
post = df_post.index

In [ ]:
df_post = df[df.index.isin(post)]

target_post = pd.DataFrame(df_post.NTEE_CD)
data_post = df_post.drop(['NTEE_CD', 'NAME_FULL', 'STREET', 'ZIP', 'CITY','ACTIVITY', 'EIN', 'ACT1', 'ACCT_PD'], axis=1)

In [ ]:
data_post.isna().sum()

In [ ]:
target_full = pd.DataFrame(df.NTEE_CD)
data_full = df.drop(['NTEE_CD', 'NAME_FULL', 'STREET', 'ZIP', 'CITY','ACTIVITY', 'EIN', 'ACT1', 'ACCT_PD'], axis=1)

In [ ]:
data_full.info(), target_full.head()

In [ ]:
data_full.shape

---
## MODELS 

Decision Tree<a id="clf_DT"></a>  
[TOP](#TOP)

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

Training sets <a id='TTS'></a>  
[TOP](#TOP)

In [ ]:
df_dum_post = pd.get_dummies(data_post, columns=['STATE', 'GROUP', 'SUBSECTION', 'AFFILIATION', 
                                       'CLASSIFICATION', 'DEDUCTIBILITY', 'FOUNDATION', 
                                       'ORGANIZATION', 'STATUS', 'FILING_REQ_CD', 
                                       'PF_FILING_REQ_CD'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train_post, X_test_post, y_train_post, y_test_post = train_test_split(df_dum_post, target_post, test_size = .2)

In [ ]:
X_train_post.to_pickle('X_train_post')
X_test_post.to_pickle('X_test_post')
y_train_post.to_pickle('y_train_post')
y_test_post.to_pickle('y_test_post')

In [ ]:
import matplotlib as plt
import pandas as pd
import numpy as np

X_train_post = pd.read_pickle('X_train_post')
y_train_post = pd.read_pickle('y_train_post')
X_test_post = pd.read_pickle('X_test_post')
y_test_post = pd.read_pickle('y_test_post')

In [ ]:
X_train_post.shape

___

In [ ]:
df_dum_full = pd.get_dummies(data_full, columns=['STATE', 'GROUP', 'SUBSECTION', 'AFFILIATION', 
                                       'CLASSIFICATION', 'DEDUCTIBILITY', 'FOUNDATION', 
                                       'ORGANIZATION', 'STATUS', 'FILING_REQ_CD', 
                                       'PF_FILING_REQ_CD'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(df_dum_full, target_full, test_size = .2)

In [ ]:
X_train_full.to_pickle('X_train_full')
X_test_full.to_pickle('X_test_full')
y_train_full.to_pickle('y_train_full')
y_test_full.to_pickle('y_test_full')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

X_train_full = pd.read_pickle('X_train_full')
y_train_full = pd.read_pickle('y_train_full')
X_test_full = pd.read_pickle('X_test_full')
y_test_full = pd.read_pickle('y_test_full')

In [ ]:
X_train_full.info()

In [ ]:
X_train.shape, y_train.shape

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import time

clf_DT = DecisionTreeClassifier(max_features='auto', max_depth=10, min_samples_split=10000)

In [ ]:
start_time = time.time()

clf_DT.fit(X_train_full, y_train_full)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
X_train_post.isna().sum()

In [ ]:
y_hat = clf_DT.predict(X_test_full)

In [ ]:
acc = accuracy_score(y_test_full,y_hat) * 100
print("Accuracy is :{0}".format(acc))

In [ ]:
from yellowbrick.classifier import ConfusionMatrix
fig, ax = plt.subplots(figsize=(18,12)) #Making the final out put a bit more readable than the standard
cm = ConfusionMatrix(clf_DT, ax=ax) #This instance takes in an axes to add to a exisiting figure.

cm.score(X_test_full, y_test_full)

# How did we do?
cm.poof()

In [ ]:
df_full.NTEE_CD.value_counts()

In [ ]:
df_full.NTEE_CD.value_counts(normalize=True)

My classes are imbalanced. I forgot to review my target variables to ensure they were correct. BUT I got ~20% accuracy, which makes sense since that is my biggest class. This is a baseline, even a very rough one. There were a few classes where most everything ended up, which is intriguing within itself. 

In [ ]:
removal = ['c', '0', '8'] #Setting what we don't want
mask = df_full[df_full.NTEE_CD.isin(removal)] #Finding the rows that contain those values and masking a mask
df_full = df_full.drop(mask.index) #Dropping based on index
df_full.shape

Now I am going to check the top 5 percentages of the groups within all my columns. I want to understand why B and P were such popular columns. Also how weighted my classes may be.

In [ ]:
df_full.NT